## Create project

This notebook creates a project for OpenSfM to run a reconstruction on. Prerequisites:
- Create a directory called projects/ within the current directory
- Within projects/, create a folder with whatever name you want to give to your current reconstruction
- Within this individual project, download a geojson called map.geojson with your AOI from https://geojson.io/

In [24]:
# downloading the metadata file if it does not exist
import os
if not os.path.isfile("ladi_images_metadata.csv"):
    !wget http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv

In [25]:
# importing the usual things
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json
import shutil

# for downloading images
import requests

# importing the geoprocessing libraries
import geopandas as gpd
import fiona # library for reading/writing GIS files, comes w/ geopandas
from shapely.geometry import Point, LineString, Polygon

In [32]:
# change parameters for individual projects
target_date_start = "2016-08-01"
target_date_end = "2017-09-30"
project_path = "projects/baton_rouge/"

In [33]:
# load the csv as a gdf
df = pd.read_csv("ladi_images_metadata.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)
df = df[df.index > target_date_start]
# df = df[df.index < target_date_end]
df = df.sort_index()
# df = df.loc[target_date_start:target_date_end]


In [34]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.gps_lon, df.gps_lat))
gdf.crs = 'epsg:4326'

geojson = gpd.read_file(project_path + 'map.geojson')
print(geojson)
poly = geojson.iloc[0, 1]
print(geojson)

          NAME                                           geometry
0  BATON ROUGE  POLYGON ((-91.19261 30.49411, -91.19267 30.494...
          NAME                                           geometry
0  BATON ROUGE  POLYGON ((-91.19261 30.49411, -91.19267 30.494...


In [35]:
# filter out images that are outside of the geojson
sample = gdf[gdf.geometry.apply(lambda x: x.within(poly))]
print(sample)

                                                         uuid    gps_lat  \
timestamp                                                                  
2016-08-14 10:25:25  363e62ad6b6fadbecaa973b8107dfa5cc97c0913  30.510122   
2016-08-14 10:25:44  c4660c2eeec595acfadeadfc814a8dc9e6abbb3d  30.508188   
2016-08-14 10:25:46  e3d8401e46eb6a440fd11653f7ce8d3cccce3e96  30.507962   
2016-08-14 10:25:47  cc712a6824c2e00585abf710ba990a8e009ff2a4  30.507840   
2016-08-14 10:25:51  e828fa3ffa1b947d4459adb51f1371e0c90f3176  30.507173   
...                                                       ...        ...   
2016-08-16 09:45:03  0d68c858057093f1ee683ff3ccc077c8312ee8d0  30.490680   
2016-08-16 09:45:05  974f57adba0ef9f240179235927037732148c8f8  30.490492   
2016-08-16 09:45:08  f66e30086114a52a9d8c7f6bd9611528a5a65a56  30.490333   
2016-08-16 09:45:10  2e16303f7709d3c46753ea911070d900a0e6cfac  30.490055   
2016-08-16 09:45:12  fbb9b5c04864dfb9db8abb4b6fc47f5a2c055bdd  30.489922   

           

In [36]:
def create_project(sample, project_path):
    # copying the jupyter notebook file
    try:
        shutil.copy("run_reconstruction_dir.ipynb", project_path)
    except IOError as e:
        print("Unable to copy file. %s" % e)
    except:
        print("Unexpected error:", sys.exc_info())
    
    # copying the config file
    try:
        shutil.copy("config.yaml", project_path)
    except IOError as e:
        print("Unable to copy file. %s" % e)
    except:
        print("Unexpected error:", sys.exc_info())
    
    # get the list of urls to download
    url_list = sample["url"].to_list()
    
    # create the images folder if it does not currently exist
    image_folder_path = project_path + "images/"
    if not os.path.exists(image_folder_path):
        os.mkdir(image_folder_path)
        
    # iterate over the images, download them all
    for url in url_list:
        image_name = url.split("/")[-1]
        image_path = image_folder_path + image_name
        if os.path.isfile(image_path):
            continue
        try:
            r = requests.get(url)
        except:
            continue
        with open(image_path, 'wb') as f:
            f.write(r.content)

In [37]:
create_project(sample, project_path)